In [2]:
import os
from urllib.request import urlretrieve
import numpy as np
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.llms import HuggingFacePipeline
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.document_loaders import UnstructuredExcelLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [3]:
from langchain_community.llms import HuggingFaceHub

'''
请注意，这里对于一般的机器，内存要求极高，我再colab尝试下载中文LLM的baichuan，但是仍然OOM，所以用的一个在线LLM做测试，如果在一些网络限制比较
严格的地方，需要开发者自己从开源社区提前下载好对应的LLM
'''
# huggingface_api_token = os.getenv('HUGGINGFACE_API_TOKEN')

# hf = HuggingFaceHub(
#     huggingfacehub_api_token=huggingface_api_token,
#     repo_id="mistralai/Mistral-7B-v0.1",
#     model_kwargs={"temperature":0.1, "max_length":2000})

# query = """电力市场与普通商品市场的差异"""  # Sample question, change to other questions you are interested in.
# hf.invoke(query)

'\n请注意，这里对于一般的机器，内存要求极高，我再colab尝试下载中文LLM的baichuan，但是仍然OOM，所以用的一个在线LLM做测试，如果在一些网络限制比较\n严格的地方，需要开发者自己从开源社区提前下载好对应的LLM\n'

In [4]:
# from zhipuai_llm import ZhipuAILLM
# zhipuai_api_key = os.getenv("ZHIPUAI_API_KEY")

# zhipuai_model = ZhipuAILLM(model='glm-4', temperature=0.1, api_key=zhipuai_api_key)
# res = zhipuai_model("电力市场与普通商品市场的差异")
# print(res)

In [5]:
from langchain_community.llms import BaichuanLLM

# Load the model
baichaun = BaichuanLLM()

res = baichaun.invoke("电力市场与普通商品市场的差异")
print(res)

电力市场与普通商品市场的差异主要体现在以下几个方面：

1. 产品特性：电力是一种特殊的商品，其生产和消费必须同时进行，无法储存。而普通商品如食品、服装等，可以在生产后储存一段时间再进行销售。

2. 价格形成机制：电力市场的价格受到供需关系、政策因素、季节性因素等多种因素影响，波动较大。而普通商品市场的价格相对稳定，受供需关系影响较小。

3. 市场参与者：电力市场的参与者主要包括发电企业、电网企业、售电企业和电力用户。而普通商品市场的参与者包括生产商、经销商、零售商和消费者。

4. 交易方式：电力市场的交易方式主要有现货交易、远期交易、竞价交易等。而普通商品市场的交易方式主要有现货交易、期货交易、拍卖等。

5. 市场监管：电力市场受到政府监管部门的严格监管，以确保电力供应的稳定和安全。而普通商品市场的监管相对宽松，主要依靠市场竞争机制来调节。

6. 环保要求：电力市场对环保要求较高，需要满足清洁能源政策的要求。而普通商品市场对环保要求相对较低。

7. 投资回报：电力市场的投资回报周期较长，风险较高。而普通商品市场的投资回报周期相对较短，风险较低。

总之，电力市场与普通商品市场在多个方面存在差异，需要针对不同市场特点进行分析和决策。


In [6]:
data = "../demo/"
# 这里为了做演示，随便找了一个in house 文本/content/0858能源动力-博士专业学位创新成果具体标准2024(最终).pdf

In [7]:
loader = PyPDFDirectoryLoader(data)

docs_before_split = loader.load()
# 过滤目录和附录
docs_before_split = [doc for doc in docs_before_split if doc.metadata['page'] > 7 and doc.metadata['page'] <275] 



In [8]:
# 这里可以理解为对自己的文档切分成块，chuck size是每一块的大小，可以根据需求调整
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 256,
    chunk_overlap  = 30,
)
docs_after_split = text_splitter.split_documents(docs_before_split)

docs_after_split[0]

Document(page_content='第一章 电力现货市场基础   \n             \n1   \n第一章 \n \n \n电力现货市场基础  \n \n \n \n  \n 1. 什么是电力市场？电力市场与 普通商品市场有哪 些差异？电力市场有\n哪些特征？ \n（1）电力市场的概念。  \n我国关于电力市场的权威解释始见于《中国电力百科全书  电力系统卷（第二版） 》。\n电力市场的定义为：基于市场经济原则，电力市场的定义为基于市场经济原则，为实现', metadata={'source': '..\\demo\\电力现货市场101问.pdf', 'page': 8})

In [9]:
from langchain_core.documents import Document

# # 提取所有文本
# full_text=""
# for doc in docs_before_split:
#     full_text += doc.page_content

# # 按段落分割
# paragraphs = full_text.split('。  \n')
# #去除段落中的空格和换行
# paragraphs = [paragraph.replace('\n','').replace(' ','') for paragraph in paragraphs]
# #去除一些无用段落
# paragraphs = [paragraph for paragraph in paragraphs if len(paragraph)>50]
# docs_after_split = [Document(page_content=doc,  metadata={"source": "..\demo\电力现货市场101问.pdf"}) for doc in paragraphs]

# docs_after_split[0]


In [10]:
print(len(docs_after_split))

1604


In [11]:
# 从过往工作经验看，embedding对于rag效果影响比较大，一般首选还是openi embedding做这一部分，开源的效果很一般
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="moka-ai/m3e-base",  # 使用m3e模型做embeddding
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

d:\miniconda3\envs\transformers\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\miniconda3\envs\transformers\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:
sample_embedding = np.array(huggingface_embeddings.embed_query(docs_after_split[0].page_content))
print("Sample embedding of a document chunk: ", sample_embedding)
print("Size of the embedding: ", sample_embedding.shape)

Sample embedding of a document chunk:  [ 1.37712257e-02  1.74429808e-02 -2.37702727e-04  3.25846276e-03
 -1.11134853e-02 -5.02367280e-02  2.90154922e-03  3.25411595e-02
  2.19006725e-02  3.71452235e-02  3.18695977e-03  1.24770182e-03
  4.05377559e-02  4.33757855e-03 -3.95487696e-02  1.82692129e-02
  4.17151116e-02  2.03029253e-02 -2.18567606e-02 -5.04174978e-02
  1.25894435e-02  1.75281409e-02 -5.76834492e-02  3.56537737e-02
 -8.49017780e-03  4.37619202e-02  3.79183562e-04  2.54495046e-03
  1.33169312e-02  3.38649489e-02  4.61364053e-02  2.00382937e-02
  9.18099657e-03  2.62288551e-04 -1.53047377e-02  3.16493586e-02
  3.07638803e-03  1.26779713e-02  5.01489341e-02 -1.27106812e-02
 -5.61213586e-03  4.18933257e-02 -3.06361988e-02 -1.15682175e-02
 -1.85013153e-02  1.92130972e-02  7.05184974e-03  3.48129943e-02
 -6.84421957e-02 -1.49334949e-02  6.85118372e-03  4.00301725e-01
 -3.31872217e-02  8.70914664e-03 -4.73613441e-02  5.77885956e-02
 -3.58327813e-02 -2.35927384e-02 -1.05876913e-02 -1

In [13]:
# 这一步是对自己前一步切好的文本做embedding处理，并通过faiss存储，faiss的主要作用是提高检索效率
vectorstore = FAISS.from_documents(docs_after_split, huggingface_embeddings)

In [15]:
query = """什么是电力金融市场？"""
         # Sample question, change to other questions you are interested in.
relevant_documents = vectorstore.similarity_search(query,k=25)
for i, doc in enumerate(relevant_documents):
    print(f"检索到的第{i+1}个内容: \n {doc.page_content}", end="\n-----------------------------------------------------\n")


# 从这一步看，已经很清晰rag的原理了，即从大量文件索引找到top k相关的text块，供下一步LLM查找、总结答案

检索到的第1个内容: 
 市场”的概念，即在狭义电力市场的基础上，将电力市场化改革领域拓展到规划、投融资、碳交易、环境约束等环节，以实现电力生产、交换相关上下游环节的全链条市场化。  
电力市场作为电力工业市场化改革的目标模式，因其改革的动因和目标取向不同，以
及市场外部环境的差异，呈现出显著的多样性，也使其结构显得格外复杂。世界各地的电力市场，既没有统一的标准供引用，也没有通用的模板可复制，需要结合各自的国情、
民情和网情建设和运营。总体上，电力市场有批发市场（ wholesale market ）与零售市场
-----------------------------------------------------
检索到的第2个内容: 
 金融输电权市场是必不可少的。  
电力市场体系是一个十分庞大的系统， 涉及电力工业的各个环节和电力经济的各个方
面，推进电力市场建设，需要规划电力市场体系建设的总体方案，通过顶层设计，分层
组织、分步实施，协调推进，以实现能源革命的战略目标。  
 3. 电力市场有哪些划分维度？具体如何划分？ 
如前所述，电力市场体系实质是电力市场交易体系，包括市场主体、交易对象、交易
类型、价格形成机制等方面。完备的电力市场通常由多个部分（子市场）共同构成，各
-----------------------------------------------------
检索到的第3个内容: 
 （2）市场性质。  
电力市场按其市场性质可分为实物市场与金融市场。一般而言，实物市场与金融市场
可以通过按产品类型和市场主体的意图两个方面加以辨识。电力实物市场建设几乎是各
国各地电力市场建设的重心，建设运营中普遍接受电力行政主管部门或监管机构的监管；电力金融市场严格意义上要接受金融监管机构的监管。  
1）电力实物市场。  
实物市场，业界也有译之为物理市场（ physical market ）的，它是以电能量及其相关
服务产品交割为目的的各类细分市场的总和，包含电力生产、传输等环节相关的自然资
-----------------------------------------------------
检索到的第4个内容: 
 义的电力市场有着明确的地域和容量指向，电力市场的地理边界可能差异很大，例如单一州（省）域内的电力市场，或者整个国家的电力市

In [21]:
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun

from typing import List
from sentence_transformers import CrossEncoder

""" 使用自定义的检索器,对上面相关性检索召回的文本块进行重排"""
class CustomRetriever(BaseRetriever):
    
    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        cross_encoder = CrossEncoder("BAAI/bge-reranker-base", max_length=512, device="cpu")
        reranked_docs = cross_encoder.rank(
        query,
        [doc.page_content for doc in relevant_documents],
        top_k=3,
        return_documents=True,)
        reranking_relevant_documents= []
        for doc in reranked_docs:
           reranking_relevant_documents.append(Document(page_content=doc["text"],metadata=relevant_documents[doc["corpus_id"]].metadata)) 
        return reranking_relevant_documents


In [12]:
# from langchain_core.retrievers import BaseRetriever
# from langchain_core.callbacks import CallbackManagerForRetrieverRun

# from typing import List
# from sentence_transformers import CrossEncoder

# """ 使用自定义的检索器,对上面相关性检索召回的文本块进行重排"""
# class CustomRetriever(BaseRetriever):
    
#     def _get_relevant_documents(
#         self, query: str, *, run_manager: CallbackManagerForRetrieverRun
#     ) -> List[Document]:
#         cross_encoder = CrossEncoder("cross-encoder/ms-marco-TinyBERT-L-2-v2", max_length=512, device="cpu")
#         reranked_docs = cross_encoder.rank(
#         query,
#         [doc.page_content for doc in relevant_documents],
#         top_k=3,
#         return_documents=True,)
#         reranking_relevant_documents= []
#         for doc in reranked_docs:
#            reranking_relevant_documents.append(Document(page_content=doc["text"],metadata=relevant_documents[doc["corpus_id"]].metadata)) 
#         return reranking_relevant_documents



In [22]:
retriever = CustomRetriever()

In [20]:
from sentence_transformers import CrossEncoder

cross_encoder = CrossEncoder(
    "BAAI/bge-reranker-base", max_length=512, device="cpu"
)
reranked_docs = cross_encoder.rank(
        query,
        [doc.page_content for doc in relevant_documents],
        top_k=3,
        return_documents=True,)
reranked_docs

[{'corpus_id': 0,
  'score': 0.99982136,
  'text': '品种、交易时间、竞争模式等维度进一步细分，各分类市场的知识将在后续问题讨论中详述。  \n（2）电力市场和普通商品市场的差异。  \n电力市场相较于普通商品市场具有显著的特殊性， 归根到底就在于电力商品与其他一\n般商品的差异性。电力商品本质即电能，与其他商品最本质的差异在于其自然属性和社\n会属性。  \n电力商品具有无仓储性。电能的生产、交割和消费几乎是同时完成的，其交割速度远\n快于一般商品，因此也不存在一般商品一手交钱一手交货的交易方式。'},
 {'corpus_id': 7,
  'score': 0.9995147,
  'text': '交易规则对电力也适用，不需要任何特殊的交易规则和协议。其实不然，电力商品较之普通商品有其无可争议的特殊性。问题 1从商品属性回答了电力市场与普通商品市场的\n差异，要深入了解电力现货市场的特殊性，则必须深刻理解电力的自然属性，自然属性\n的电力特殊性是与电力现货市场交易规则的特殊性高度一致的。  \n1）电力具有难以储存的特殊性。这一特点导致供需双方提前达成的交易结果与需求\n方实时消费的电能量在数量上必然存在偏差，为弥补这种偏差，势必需要维持实时平衡\n的特殊的市场规则。'},
 {'corpus_id': 2,
  'score': 0.9783352,
  'text': '电力商品具有同质性。电能不带有任何生产者的标识，电能生产者将生产的电能输入'}]

In [16]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})
# 这里的K与检索文本的数量相关，但一般不会太大

In [17]:
prompt_template = """
请用下面相关文本回答问题，如果不知道答案，就回复不知道，

{context}

Question: {question}

Helpful Answer:
"""

PROMPT = PromptTemplate(
 template=prompt_template, input_variables=["context", "question"]
)

In [18]:
retrievalQA = RetrievalQA.from_chain_type(
    llm=baichaun,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

In [19]:
result = retrievalQA.invoke({"query": query})
print(result['result'])

电力金融市场是电力市场的一个子市场，它严格意义上要接受金融监管机构的监管。电力金融市场主要涉及电力商品的金融衍生品交易，如远期合约、期货合约、期权合约等，为参与主体提供了很好的价格对冲与风险管理的手段。


In [20]:
relevant_docs = result['source_documents']
print(f'There are {len(relevant_docs)} documents retrieved which are relevant to the query.')
print("*" * 100)
for i, doc in enumerate(relevant_docs):
    # print(f"Relevant Document #{i+1}:\nSource file: {doc.metadata['source']}, Page: {doc.metadata['page']}\nContent: {doc.page_content}")
    print(f"Relevant Document #{i+1}:\nSource file: {doc.metadata['source']}\nContent: {doc.page_content}")
    print("-"*100)
    print(f'There are {len(relevant_docs)} documents retrieved which are relevant to the query.')

There are 5 documents retrieved which are relevant to the query.
****************************************************************************************************
Relevant Document #1:
Source file: ..\demo\电力现货市场101问.pdf
Content: 市场”的概念，即在狭义电力市场的基础上，将电力市场化改革领域拓展到规划、投融资、碳交易、环境约束等环节，以实现电力生产、交换相关上下游环节的全链条市场化。  
电力市场作为电力工业市场化改革的目标模式，因其改革的动因和目标取向不同，以
及市场外部环境的差异，呈现出显著的多样性，也使其结构显得格外复杂。世界各地的电力市场，既没有统一的标准供引用，也没有通用的模板可复制，需要结合各自的国情、
民情和网情建设和运营。总体上，电力市场有批发市场（ wholesale market ）与零售市场
----------------------------------------------------------------------------------------------------
There are 5 documents retrieved which are relevant to the query.
Relevant Document #2:
Source file: ..\demo\电力现货市场101问.pdf
Content: 金融输电权市场是必不可少的。  
电力市场体系是一个十分庞大的系统， 涉及电力工业的各个环节和电力经济的各个方
面，推进电力市场建设，需要规划电力市场体系建设的总体方案，通过顶层设计，分层
组织、分步实施，协调推进，以实现能源革命的战略目标。  
 3. 电力市场有哪些划分维度？具体如何划分？ 
如前所述，电力市场体系实质是电力市场交易体系，包括市场主体、交易对象、交易
类型、价格形成机制等方面。完备的电力市场通常由多个部分（子市场）共同构成，各
--------------------------------------------------------------

In [ ]:
'''
fine tuning和rag还是要经过实验对比，具体看效果，在LLM之前，大家比较习惯都先fine tuning下（比如BERT系列），但从上面仅仅是下载模型，
colab都OOM了看，fine tuning对于机器的要求更高，而且数据较少的情况下效果不好说。

也可以fine tuning之后，再结合fine tuning之后的LLM做rag

'''